#Pizza Sipariş Sistemi

In [116]:
#Gerekli Kütüphanelerin import edilmesi
import csv
from datetime import datetime
from datetime import timedelta
from csv import writer

In [117]:
#Pizza classının oluşturlması.
class pizza:
  def __init__(self, name, description, cost):
    self.name = name
    self.description = description
    self.cost = cost

In [118]:
#Pizza alt sınıflarının oluşturlması.
klasik = pizza("Klasik","Klasik karışık pizza", 90)
margarita = pizza("Margarita","Sadece peynirli pizza", 75)
turk_pizza = pizza("Türk Pizzası","Türk lezzetleri pizza", 90)
sade = pizza("Sade", "Sade pizza",  100)

In [119]:
#Decorator classının oluşturulması. Pizza classının özelliklerini kullanır.
class decorator(pizza):
  def toplam_ucret(self,pizza):
    ucret = pizza.cost + self.cost
    return ucret
  def pizza_aciklama(self,pizza):
    desc = pizza.description + ' ' + self.description + ' ile hazırlanacaktır.'
    return desc

In [120]:
#Ek malzemelerin decorator alt sınıfı olarak oluşturulur.
zeytin = decorator("Zeytin","ekstra zeytin", 5)
mantar = decorator("Mantar","ekstra mantar", 10)
keci_peyniri = decorator("Keçi Peyniri","özel keçi peyniri", 15)
et = decorator("Et","marine edilmiş dana eti", 20)
sogan = decorator("Soğan","karamelize soğan", 7)
misir = decorator("Mısır","yerli mısır", 8)

In [121]:
#Menu.txt dosyası okunarak, menu değişkeni oluşturulur.
file = open("/Menu.txt", "r")
menu = file.read()

In [122]:
#Menü sözlüğü oluşturulur. Keyleri menüdeki numaralardan, valueları ise alt sınıflardan oluşur.
menu_dict = {
    "1" : klasik,
    "2" : margarita,
    "3" : turk_pizza,
    "4" : sade,
    "11" : zeytin,
    "12" : mantar,
    "13" : keci_peyniri,
    "14" : et,
    "15" : sogan,
    "16": misir
}

In [123]:
#Veri tabanına siparişlerin yazılması için oluşturulan fonksiyon.
def veritabanina_yazma(siparis):
  with open('Orders_Database.csv', 'a') as database:
    writer_object = writer(database)
    writer_object.writerow(siparis)
    database.close()

In [149]:
#Pizza seçimi için hazırlanan fonksiyon
def pizza_secimi():
  pizza_numarasi = input("Almak istediğiniz pizza ile ilgili numarayı giriniz: ")
  while True:
   try:
    if int(pizza_numarasi) in range(0,5) :
     secilen_pizza = menu_dict[pizza_numarasi]
     break
    pizza_numarasi = input("Lütfen menüde olan bir değer giriniz. Almak istediğiniz pizza ile ilgili numarayı giriniz: ")
   except ValueError:
    pizza_numarasi = input("Lütfen menüde olan bir değer giriniz. Almak istediğiniz pizza ile ilgili numarayı giriniz: ")
  return secilen_pizza

In [151]:
#Ek malzeme seçimi için hazırlanan fonksiyon
def ek_malzeme_secimi():
  malzeme_numarasi = input("Eklemek istediğiniz malzeme ile ilgili numarayı giriniz: ")
  while True:
    try:
     if int(malzeme_numarasi) in range(10,17):
      secilen_malzeme = menu_dict[malzeme_numarasi]
      break
     malzeme_numarasi = input("Lütfen menüde olan bir değer giriniz. Eklemek istediğiniz malzeme ile ilgili numarayı giriniz: ")
    except ValueError:
       malzeme_numarasi = input("Lütfen menüde olan bir değer giriniz. Eklemek istediğiniz malzeme ile ilgili numarayı giriniz: ")
  return secilen_malzeme

In [162]:
#Ödeme fonksiyonu
def odeme():
  kullanici_adi = input("Lütfen adınızı soyadınızı giriniz: ")
  kimlik_no = input("Lütfen TC kimlik numaranızı giriniz: ")
  #Kimlik numarası hane kontrolü
  while True:
   if len(kimlik_no) == 11:
     break
   kimlik_no = input("Kimlik numaranız 11 haneli olmalıdır. Lütfen TC kimlik numaranızı giriniz: ")
  kredi_karti_no = input("Lütfen kredi karti numaranızı giriniz: ")
  #Kredi kartı numarası hane kontrolü
  while True:
    if len(kredi_karti_no) == 16:
      break
    kredi_karti_no = input("Kredi kartı numaranız 16 haneli olmalıdır. Lütfen kredi karti numaranızı giriniz: ")
  sifre = input("Lütfen karta ait şifreyi giriniz: ")
  #Şifre hane kontrolü
  while True:
    if len(sifre) == 4:
      break
    sifre = input("Şifreniz 4 haneli olmalıdır. Lütfen karta ait şifreyi giriniz: ")
  #Sipariş zamanının Türkiye yerel saatine eşitlemek için 3 saat ekliyorum.
  siparis_zamani = (datetime.now() + timedelta(hours=3)).strftime("%d/%m/%y  %H:%M")
  return kullanici_adi, kimlik_no, kredi_karti_no, sifre, siparis_zamani

In [163]:
#Sipariş işlemlerinin gerçekleşeceği main fonksiyonu
def main():
  print(menu)
  #Siparişin detaylarını liste kullanarak database üzerine yazacağız. Main fonksiyon her çağrıldığında sıfırdan boş bir liste oluşturulyor.
  siparis = []
  #Sipariş bilgileri alınır.
  secilen_pizza = pizza_secimi()
  secilen_malzeme = ek_malzeme_secimi()
  tutar = secilen_malzeme.toplam_ucret(secilen_pizza)
  aciklama = secilen_malzeme.pizza_aciklama(secilen_pizza) + " Toplam tutar: " + str(tutar) +"TL"
  print(aciklama)
  #Ödeme işlemi odeme fonksiyonu çağrılarak gerçekleştiriliyor.
  global kullanici_adi, kimlik_no, kredi_karti_no, sifre, siparis_zamani
  kullanici_adi, kimlik_no, kredi_karti_no, sifre, siparis_zamani = odeme()
  #Sipariş bilgileri order listesine eleman olarak eklenir.
  siparis.append(kullanici_adi)
  siparis.append(kimlik_no)
  siparis.append(kredi_karti_no)
  siparis.append(sifre)
  siparis.append(aciklama)
  siparis.append(siparis_zamani)
  #Sipariş bilgileri veritabanına yazılır.
  veritabanina_yazma(siparis)
  print("Siparişiniz başarı ile alınmıştır.")

In [165]:
main()

* Lütfen Bir Pizza Tabanı Seçiniz:
1: Klasik
2: Margarita
3: TürkPizza
4: Sade Pizza
* ve seçeceğiniz sos:
11: Zeytin
12: Mantarlar
13: Keçi Peyniri
14: Et
15: Soğan
16: Mısır
* Teşekkür ederiz!

Almak istediğiniz pizza ile ilgili numarayı giriniz: 2
Eklemek istediğiniz malzeme ile ilgili numarayı giriniz: 15
Sadece peynirli pizza karamelize soğan ile hazırlanacaktır. Toplam tutar: 82TL
Lütfen adınızı soyadınızı giriniz: Joey Tribianni
Lütfen TC kimlik numaranızı giriniz: 77778888999
Lütfen kredi karti numaranızı giriniz: 1111222244445555
Lütfen karta ait şifreyi giriniz: 7777
Siparişiniz başarı ile alınmıştır.


In [166]:
main()

* Lütfen Bir Pizza Tabanı Seçiniz:
1: Klasik
2: Margarita
3: TürkPizza
4: Sade Pizza
* ve seçeceğiniz sos:
11: Zeytin
12: Mantarlar
13: Keçi Peyniri
14: Et
15: Soğan
16: Mısır
* Teşekkür ederiz!

Almak istediğiniz pizza ile ilgili numarayı giriniz: 2
Eklemek istediğiniz malzeme ile ilgili numarayı giriniz: 12
Sadece peynirli pizza ekstra mantar ile hazırlanacaktır. Toplam tutar: 85TL
Lütfen adınızı soyadınızı giriniz: Rachel Green
Lütfen TC kimlik numaranızı giriniz: 33333555111
Lütfen kredi karti numaranızı giriniz: 1111444477778888
Lütfen karta ait şifreyi giriniz: 2222
Siparişiniz başarı ile alınmıştır.
